In [ ]:
# %%
from matplotlib import pyplot as plt

from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize

import pandas as pd
import numpy as np
import csv
import os
from scipy import sparse
from scipy.sparse.linalg import spsolve
from scipy.signal import savgol_filter

import warnings
warnings.filterwarnings('ignore',category=DeprecationWarning)
inputdf=pd.read_csv("AD_2023_processed.csv")
wavenumbermin=900
wavenumbermax=2000
wavenumberpoints=701
wavenumber=[float(i) for i in inputdf.columns[10:].values]
#wavenumbermax=900+241
#wavenumberpoints=242

#"Asymmetric Least Squares Smoothing" by P. Eilers and H. Boelens in 2005
def baseline_als(y, lam=10000, p=0.0001, niter=10):
    L = len(y)
    D = sparse.diags([1,-2,1],[0,-1,-2], shape=(L,L-2))
    D = lam * D.dot(D.transpose()) # Precompute this term since it does not depend on `w`
    w = np.ones(L)
    W = sparse.spdiags(w, 0, L, L)
    for i in range(niter):
        W.setdiag(w) # Do not create a new matrix, just update diagonal values
        Z = W + D
        z = spsolve(Z, w*y)
        w = p * (y > z) + (1-p) * (y < z)
    return z

def moremetrics(trainner, X, y,classes):
    #print precision recall f1
    scores = cross_validate(trainner, X, y, cv=5,scoring=('precision','recall','f1'))
    print("validation precision: %", float(sum(scores['test_precision'])/len(scores['test_precision'])*100))
    print("validation recall: %", float(sum(scores['test_recall'])/len(scores['test_recall'])*100))
    print("validation f1: %", float(sum(scores['test_f1'])/len(scores['test_f1'])*100))
    #print(confusion_matrix(y_test,y_pred))
    #print(classification_report(y_test,y_pred))
    #confusion matrix
    return classes+[float(sum(scores['test_precision'])/len(scores['test_precision'])*100),\
            float(sum(scores['test_recall'])/len(scores['test_recall'])*100),\
            float(sum(scores['test_f1'])/len(scores['test_f1'])*100)]

def f_importances(coef, intercept, names,classes,svm_C,svmcrossacc,i):
    with open('./outputs/svm_feature_importance.csv','a', newline='',encoding='utf-8') as csvfile:
        spamwriter = csv.writer(csvfile)
        if i ==0:
            spamwriter.writerow([i]+['class']*len(classes)+['crossacc']
                                +['class']*len(classes)+['C']+['intercept']+names)
        coef=coef.tolist()
        spamwriter.writerows([[i]+classes+svmcrossacc+[svm_C]+[str(i) for i in intercept.tolist()]+[str(k) for k in j] for j in coef])
    plt.figure()
    plt.plot(names,coef[0])
    plt.ylabel('svm_feature importance')

def doUMAP(column,classes,X,y):

    #plot UMAP plot plt.figure(1)
        
    X=StandardScaler().fit_transform(X)

    y = y[column].values.tolist()
    
    import umap
    reducer = umap.UMAP(random_state=42)
    
    reducer = umap.UMAP(a=None, angular_rp_forest=False, b=None,
     force_approximation_algorithm=False, init='spectral', learning_rate=1.0,
     local_connectivity=1.0, low_memory=False, metric='euclidean',
     metric_kwds=None, min_dist=0.1, n_components=2, n_epochs=None,
     n_neighbors=15, negative_sample_rate=5, output_metric='euclidean',
     output_metric_kwds=None, random_state=42, repulsion_strength=1.0,
     set_op_mix_ratio=1.0, spread=1.0, target_metric='categorical',
     target_metric_kwds=None, target_n_neighbors=-1, target_weight=0.5,
     transform_queue_size=4.0, transform_seed=42, unique=False, verbose=False)

    reducer.fit(X)
    embedding = reducer.transform(X)
    
  
    target={'target':y}
    index={'index':[]}
    target = pd.DataFrame(data=target)
    index=pd.DataFrame(data=index)

    principalDf = pd.DataFrame(data = embedding, columns = ['Embedding 1', 'Embedding 2'])
    finalDf = pd.concat([principalDf, target], axis=1)

    fig = plt.figure(figsize = (8,8))
    fig.suptitle('UMAP')
    ax = fig.add_subplot(1,1,1) 
    ax.set_xlabel('Embedding 1', fontsize = 15)
    ax.set_ylabel('Embedding 2', fontsize = 15)
    ax.set_title('2 component UMAP', fontsize = 20)
    targets=classes
    colors = ['C0','C1','C2','C3','C4','C5','C6','C7','C8',
              'C9','C10','C11','C12','C13']
    for target, color in zip(targets,colors):
        indicesToKeep = finalDf['target'] == target
        ax.scatter(finalDf.loc[indicesToKeep, 'Embedding 1']
                   , finalDf.loc[indicesToKeep, 'Embedding 2']
                   , c = color
                   , s = 10)
    ax.legend(targets)
    ax.grid()
    #plt.xlim(-50, 50)
    #plt.ylim(-50, 50)
    #save UMAP figure
    plt.savefig("./outputs/"+column+"UMAP.png")

def doPCA(filename,column,classes,X,y):
    #exclude grahene peak
    #wavenumber=wavenumber[:1233]+wavenumber[1355:]
           

    y = y[column].values.tolist()

    #plot PCA plot plt.figure(1)
        
    #X=StandardScaler().fit_transform(X)
    
    target={'target':y}
    index={'index':[]}
    target = pd.DataFrame(data=target)
    index=pd.DataFrame(data=index)

    pca = PCA(n_components=3)
    principalComponents = pca.fit_transform(X)[:,0:3]
    pca.components_[0]
    principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2', 'principal component 3'])
    finalDf = pd.concat([principalDf, target], axis=1)
    fig = plt.figure(figsize = (8,8))
    fig.suptitle('PCA')
    ax = fig.add_subplot(1,1,1,projection='3d') 
    ax.set_xlabel('Principal Component 1', fontsize = 15)
    ax.set_ylabel('Principal Component 2', fontsize = 15)
    ax.set_zlabel('Principal Component 3', fontsize = 15)
    ax.set_title('2 component PCA', fontsize = 20)
    targets=classes
    colors = ['C0','C1','C2','C3','C4','C5','C6','C7','C8',
              'C9','C10','C11','C12','C13']
    for target, color in zip(targets,colors):
        indicesToKeep = finalDf['target'] == target
        #ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
        #           , finalDf.loc[indicesToKeep, 'principal component 2']
        #           , c = color
        #           , s = 10)
        ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1'],
                   finalDf.loc[indicesToKeep, 'principal component 2'],
                   finalDf.loc[indicesToKeep, 'principal component 3'],
                   c = color,
                   s = 10)
    ax.legend(targets)
    ax.grid()
    #plt.xlim(-50, 50)
    #plt.ylim(-50, 50)
    print('pc1 pc2 explains',pca.explained_variance_ratio_)
    #save PCA figure
    plt.savefig("./outputs/"+filename+column+"PCA.png")
    
    plt.figure()
    plt.plot(wavenumber,pca.components_[0], label = "PC1")
    plt.plot(wavenumber,pca.components_[1], label = "PC2")
    plt.legend()
    plt.savefig("./outputs/"+filename+column+"PC.png")

    csv_output=np.zeros((len(targets)+1,len(wavenumber)))
    csv_output[0,:]=wavenumber
    plt.figure()
    shift=0
    for target, color in zip(targets,colors):
        plottarget=finalDf[finalDf['target']==target]
        plotX=X[finalDf['target']==target,:]
        plt.plot(wavenumber,np.mean(plotX,axis=0)+shift,c=color,label=target)
        shift+=1
        csv_output[shift,:]=np.mean(plotX,axis=0)
    plt.legend()
    plt.savefig("./outputs/"+filename+column+"spectra.png")
    np.savetxt('./outputs/'+filename+column+"spectra.csv", csv_output, delimiter=',')
    
    
def doML(MLdf,column,classes,standardize=False,moremetrices=True,backgroundsub=False,svm_C=1e-10,max_iter=400,normalizepeak=475):
    #exclude grahene peak
    #wavenumber=wavenumber[:1233]+wavenumber[1355:]
    X=[]
    y=[]
    folder=[]
    outaccuracy=[]
    
    outputname="_".join([str(i) for i in classes])


    df=MLdf.loc[MLdf[column].isin(classes)]
    if MLdf.loc[MLdf[column].isin(classes)].size==0:
        print(substrate+process+'not found')
        return
      
    X = np.array(df.iloc[:,10:])
    def reject_outliers(data,y, m=3):
        maximums=np.max(data,axis=1)
        index=abs(maximums - np.mean(maximums)) < m * np.std(maximums)
        return data[index,:],y[index]
    X,df=reject_outliers(X,df)
    X=savgol_filter(X, 7, 2)

    rowtodelete=np.max(X[:,normalizepeak-5:normalizepeak+5],axis=1)>0.1 #not dry samples
    #rowtodelete=np.logical_and(np.max(X[:,normalizepeak-5:normalizepeak+5],axis=1)>0.1,
    #                           np.max(X,axis=1)<=2*np.max(X[:,normalizepeak-5:normalizepeak+5],axis=1))

    X=X[rowtodelete,:]
    df=df.iloc[rowtodelete]

    for i in range(np.shape(X)[0]):
        #normalize 1661
        peak=max(X[i,normalizepeak-5:normalizepeak+5])
        if peak !=0:
            X[i,:]=X[i,:]/peak

    codes, uniques=pd.factorize(df[column])
    y = np.array(codes)
    classes=uniques.tolist()

    if backgroundsub:
        for i in range(np.shape(X)[0]):
            X[i,:]=X[i,:]-baseline_als(X[i,:])

    X_trainori, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
    #standardize
    if standardize:
        scaler = StandardScaler()
        scaler.fit(X_trainori)
        X_train=scaler.transform(X_trainori)
        X_test=scaler.transform(X_test)
        #X[~y.astype(bool)]=StandardScaler().fit_transform(X[~y.astype(bool)])
        #X[y.astype(bool)]=StandardScaler().fit_transform(X[y.astype(bool)])
        #X=normalize(X, norm='l2')
    else:
        X_train=X_trainori


    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
    strat_k_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=12138)

    crossscaler = StandardScaler()
    #train svm on raw data
    print("svm train on raw",len(y_train),"test",len(y_test))
    svmtrainner = svm.LinearSVC(multi_class='ovr',C=svm_C,max_iter=max_iter)
    if standardize: 
        scores = cross_val_score(Pipeline([('transformer', crossscaler), ('estimator', svmtrainner)]), X, y, cv=strat_k_fold)
    else:
        scores = cross_val_score(Pipeline([('estimator', svmtrainner)]), X, y, cv=strat_k_fold)
    svmtrainner.fit(X_train, y_train)
    y_pred = svmtrainner.predict(X_test)
    svmcrossacc=sum(scores)/len(scores)*100
    print("validation scores: %", float(sum(scores)/len(scores)*100), scores)
    if moremetrices:
        outaccuracy.append(moremetrics(svmtrainner, X, y,classes))

    #train logistics regression on raw data
    print("logistics regression train on raw",len(y_train),"test",len(y_test))
##    parameters = {'estimator__C':[1e-1,1e-2,1e-3,1e-4],
##                  'estimator__l1_ratio':[0.1,0.2,0.3,0.4,0.5,0.6]}
    parameters = {'estimator__C':[svm_C],
                  'estimator__l1_ratio':[0.3]}
    logisticstrainner = LogisticRegression(solver='saga',penalty='elasticnet',max_iter=max_iter)
    if standardize:    
        clf = GridSearchCV(Pipeline([('transformer', crossscaler), ('estimator', logisticstrainner)]), parameters)
    else:
        clf = GridSearchCV(Pipeline([('estimator', logisticstrainner)]), parameters)
    clf.fit(X_train, y_train)
    print(clf.best_params_)
    print(clf.score(X_test, y_test))
    logisticstrainner = LogisticRegression(solver='saga',max_iter=max_iter,C=clf.best_params_['estimator__C']
                                           ,penalty='elasticnet',l1_ratio=clf.best_params_['estimator__l1_ratio'])
    if standardize: 
        scores = cross_val_score(Pipeline([('transformer', crossscaler), ('estimator', logisticstrainner)]), X, y, cv=strat_k_fold)
    else:
        scores = cross_val_score(Pipeline([('estimator', logisticstrainner)]), X, y, cv=strat_k_fold)        
    logisticstrainner.fit(X_train, y_train)
    y_pred = logisticstrainner.predict(X_test)
    logcrossacc=sum(scores)/len(scores)*100
    print("validation scores: %", float(sum(scores)/len(scores)*100), scores)
    if moremetrices:
        outaccuracy.append(moremetrics(logisticstrainner, X, y,classes))

    #train Decision tree on raw data
    print("Decision tree train on raw",len(y_train),"test",len(y_test))
    decisiontrainner = DecisionTreeClassifier()
    if standardize:
        scores = cross_val_score(Pipeline([('transformer', crossscaler), ('estimator', decisiontrainner)]), X, y, cv=strat_k_fold)
    else:
        scores = cross_val_score(Pipeline([('estimator', decisiontrainner)]), X, y, cv=strat_k_fold)        
    decisiontrainner.fit(X_train, y_train)
    y_pred = decisiontrainner.predict(X_test)
    print("validation scores: %", float(sum(scores)/len(scores)*100), scores)
    if moremetrices:
        outaccuracy.append(moremetrics(decisiontrainner, X, y,classes))
    
    #train Random forest on raw data
    print("Random forest train on raw",len(y_train),"test",len(y_test))
    randomforesttrainner = RandomForestClassifier(n_estimators=100)
    if standardize:
        scores = cross_val_score(Pipeline([('transformer', crossscaler), ('estimator', randomforesttrainner)]), X, y, cv=strat_k_fold)
    else:
        scores = cross_val_score(Pipeline([('estimator', randomforesttrainner)]), X, y, cv=strat_k_fold)        
    randomforesttrainner.fit(X_train, y_train)
    y_pred = randomforesttrainner.predict(X_test)
    print("validation scores: %", float(sum(scores)/len(scores)*100), scores)
    if moremetrices:
        outaccuracy.append(moremetrics(randomforesttrainner, X, y,classes))

    #XGBoost
    from xgboost import XGBClassifier
    print("XGBoost train on raw",len(y_train),"test",len(y_test))
    XGBoosttrainner = XGBClassifier()
    if standardize:
        scores = cross_val_score(Pipeline([('transformer', crossscaler), ('estimator', XGBoosttrainner)]), X, y, cv=strat_k_fold)
    else:
        scores = cross_val_score(Pipeline([('estimator', XGBoosttrainner)]), X, y, cv=strat_k_fold)        
    XGBoosttrainner.fit(X_train, y_train)
    y_pred = XGBoosttrainner.predict(X_test)
    print("validation scores: %", float(sum(scores)/len(scores)*100), scores)
    if moremetrices:
        outaccuracy.append(moremetrics(XGBoosttrainner, X, y,classes))

    #non-negative sparse logistic regression
##    from glmnet import LogitNet
##    print("non-negative logistic train on raw",len(y_train),"test",len(y_test))
##    breakpoint()
##    Logittrainner = LogitNet()
##    scores = cross_val_score(Pipeline([('transformer', crossscaler), ('estimator', Logittrainner)]), X, y, cv=5)
##    Logittrainner.fit(X_train, y_train)
##    y_pred = Logittrainner.predict(X_test)
##    Logitcrossacc=sum(scores)/len(scores)*100
##    print("validation scores: %", float(sum(scores)/len(scores)*100), scores)
##    if moremetrices:
##        outaccuracy.append(moremetrics(Logittrainner, X, y,classes))
        
    #plot PCA plot plt.figure(1)
    target={'target':y}
    index={'index':[]}
    target = pd.DataFrame(data=target)
    index=pd.DataFrame(data=index)

    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(X)
    principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2'])
    finalDf = pd.concat([principalDf, target], axis=1)
    finalPC_Df = pd.DataFrame(data = pca.components_[0:2,:], columns = wavenumber)

    fig = plt.figure(figsize = (8,8))
    fig.suptitle('PCA')
    ax = fig.add_subplot(1,1,1) 
    ax.set_xlabel('Principal Component 1', fontsize = 15)
    ax.set_ylabel('Principal Component 2', fontsize = 15)
    ax.set_title('2 component PCA', fontsize = 20)
    targets = [0,1]
    targetsname=" ".join([str(i) for i in classes])
    colors = ['g','r','b']
    for target, color in zip(targets,colors):
        indicesToKeep = finalDf['target'] == target
        ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
                   , finalDf.loc[indicesToKeep, 'principal component 2']
                   , c = color
                   , s = 10)
    ax.legend(targets)
    ax.grid()
    #plt.xlim(-50, 50)
    #plt.ylim(-50, 50)
    print('pc1 pc2 explains',pca.explained_variance_ratio_)
    #save PCA csv
    plt.savefig("./outputs/"+outputname+"PCA.png")
    finalDf.to_csv("./outputs/"+outputname+"PCA.csv")
    finalPC_Df.to_csv("./outputs/"+outputname+"PCA_PC.csv")
    
    plt.close("all")
    for target, color in zip(targets,colors):
        plottarget=y[y==target]
        plotX=X[y==target,:]
        plotindex=np.random.permutation(plotX.shape[0])
        plt.plot(wavenumber,plotX[plotindex[0],:],c=color,label=target)
        for i in range(20):
            plt.plot(wavenumber,plotX[plotindex[i],:],c=color)
    plt.legend()
    plt.savefig("./outputs/"+outputname+"spectra.png")
    
    #plot fig
        
    #output svm feature importance for 5 times
    for i in range(5):
        X_trainori, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
        #standardize
        if standardize:
            scaler = StandardScaler()
            scaler.fit(X_trainori)
            X_train=scaler.transform(X_trainori)
            X_test=scaler.transform(X_test)
        else:
            X_train=X_trainori
        svmtrainner = svm.LinearSVC(multi_class='ovr',fit_intercept=True,max_iter=max_iter)#,penalty='l1',dual=False)
        svmtrainner.fit(X_train, y_train)
        y_pred = svmtrainner.predict(X_test)
        print(classification_report(y_test,y_pred))
        accmatrix = confusion_matrix(y_test, y_pred)
        accbyclass=accmatrix.diagonal()/accmatrix.sum(axis=1)
        #print(svmtrainner.coef_,features_names)
        print('svm interscept is:',svmtrainner.intercept_)
        f_importances(svmtrainner.coef_, svmtrainner.intercept_, wavenumber,classes,svm_C,[svmcrossacc]+accbyclass.tolist(),i)
    #output logistic feature importance for 5 times
    for i in range(5):
        X_trainori, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
        #standardize
        if standardize:
            scaler = StandardScaler()
            scaler.fit(X_trainori)
            X_train=scaler.transform(X_trainori)
            X_test=scaler.transform(X_test)
        else:
            X_train=X_trainori
        logisticstrainner = LogisticRegression(solver='saga',max_iter=max_iter,C=clf.best_params_['estimator__C']
                                               ,penalty='elasticnet',l1_ratio=clf.best_params_['estimator__l1_ratio'],fit_intercept=True)
        logisticstrainner.fit(X_train, y_train)
        y_pred = logisticstrainner.predict(X_test)
        print(classification_report(y_test,y_pred))
        accmatrix = confusion_matrix(y_test, y_pred)
        accbyclass=accmatrix.diagonal()/accmatrix.sum(axis=1)
        #print(svmtrainner.coef_,features_names)
        print('log interscept is:',logisticstrainner.intercept_)
        f_importances(logisticstrainner.coef_,logisticstrainner.intercept_, wavenumber,classes,str(clf.best_params_),[logcrossacc]+accbyclass.tolist(),i)

        
    plt.close("all")
    #plt.plot(wavenumber,svmtrainner.coef_[0])
    #plt.show()
    if moremetrices:
        with open('./summary.csv','a', newline='',encoding='utf-8') as csvfile:
            spamwriter = csv.writer(csvfile)
            spamwriter.writerows(outaccuracy)
    #plt.show()

def doCC(CCdf,outputname='CCfigure',backgroundsub=False):
    

    X=[]

    folder=[]
    outaccuracy=[]

    X = np.array(CCdf.iloc[:,10:])
    if backgroundsub:
        for i in range(np.shape(X)[0]):
            X[i,:]=X[i,:]-baseline_als(X[i,:])
    
    pd.DataFrame(X).to_csv("./outputs/"+outputname+"_data.csv")
    
    plt.plot(wavenumber,np.average(X,axis=0))
    plt.xlim([wavenumbermin,wavenumbermax])
    plt.savefig("./outputs/"+outputname+"_average.png")
    plt.close("all")
    X=StandardScaler().fit_transform(X)

    #calculate correlation
    cor=np.dot(np.transpose(X),X)/X.shape[0]
    pd.DataFrame(cor).to_csv("./outputs/"+outputname+"_correlation.csv")
    cor[abs(cor) < 0.8] = 0
    plt.imshow(cor, cmap='hot', interpolation='nearest'
               ,extent=[wavenumbermin,wavenumbermax,wavenumbermax,wavenumbermin],
               vmin=-1, vmax=1)
    plt.colorbar()
    plt.savefig("./outputs/"+outputname+"_correlation.png")
    

C:\Users\tiger\AppData\Local\Temp\ipykernel_26016\438646873.py:33: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  inputdf=pd.read_csv("AD_2023_processed.csv")


In [ ]:
standardize=False
moremetrices=False

normalizepeak=475 #1660

print('normalized by:', wavenumber[normalizepeak])


MLindex=inputdf['side2'].isin(['right'])& \
        inputdf['adlabel'].isin([0,1])

MLdf=inputdf.loc[MLindex]



X = np.array(inputdf.iloc[:,10:].loc[MLindex])
y=inputdf.iloc[:,:10].loc[MLindex]
#set classes
column='adlabel'
classes=[0,1]

def reject_outliers(data,y, m=3):
    maximums=np.max(data,axis=1)
    index=abs(maximums - np.mean(maximums)) < m * np.std(maximums)
    return data[index,:],y[index]
#outliers remove
X,y=reject_outliers(X,y)
X=savgol_filter(X, 7, 2)


for i in range(np.shape(X)[0]):
    #normalize 1658:475,1439:333
    peak=max(X[i,normalizepeak-5:normalizepeak+5])
    if peak !=0:
        X[i,:]=X[i,:]/max(X[i,normalizepeak-5:normalizepeak+5])

print(MLdf,classes,'standardize',standardize)
doML(MLdf,column,classes,standardize,moremetrices,max_iter=10000,normalizepeak=normalizepeak)
